## generate qr codes

After scraping the top songs, use the generated links to request the corresponding qr codes.

Assign a random colour from the selection to each qr code.

The link skeleton is described in [this github issue](https://github.com/spotify/web-api/issues/519#issuecomment-618114678), the colours are from [here](https://github.com/sarraahr/limonata-colors).

In [ ]:
import json
import random
from os import makedirs
import requests

random.seed(1234)

makedirs("qr_codes",exist_ok=True)

In [ ]:
FORMAT = "png"
SIZE = 290

colours = json.load(open("colours.json")) 
song_links = json.load(open("song_links.json"))



link_skeleton = "https://scannables.scdn.co/uri/plain/{format}/{colour}/{text_colour}/{size}/spotify:track:{url}"

This is to adjust the text colour accordingly, so that the contrast is sensible.

In [ ]:
def get_text_colour(hex_color):
    # Convert hex color to RGB
    r = int(hex_color[1:3], 16)
    g = int(hex_color[3:5], 16)
    b = int(hex_color[5:7], 16)

    # Calculate the relative luminance
    # chatgpt wrote this, no clue where these numbers come from
    # but it works.
    luminance = (0.2126 * r + 0.7152 * g + 0.0722 * b) / 255

    # Choose black or white based on luminance
    return "black" if luminance > 0.5 else "white"

In [ ]:
background_colours = []
for rank, (name, link) in enumerate(song_links.items()):
    colour = random.choice(colours).replace("#", "")
    text_colour = get_text_colour(colour)
    url = link.split("track/")[1]
    qr_link = link_skeleton.format(
        format=FORMAT, colour=colour, text_colour=text_colour, size=SIZE, url=url
    )

    response = requests.get(qr_link)

    file_name = f"qr_codes/{rank}_{name}.{FORMAT}"

    with open(file_name, "wb") as f:
        f.write(response.content)

    background_colours.append(f"#{colour}")

with open("background_colours.json", "w") as f:
    json.dump(background_colours,f)